## Importing Packages

In [1]:
import pandas as pd
from itertools import combinations
from collections import Counter

## Reading data

In [2]:
# Create sample data
orders = pd.read_csv( "D:\\Academics\\Datahack\\jetson-sample-data.csv")

## Get all pairs of combinations of items

In [3]:
# Get all pairs of items
item_pairs = list(combinations(set(orders['item_name']),2 ))

## Frequency count for each item

In [4]:
# Get frequency counts for each item pair
freq_counts = Counter()
for order_id, group in orders.groupby('order_id'):
    items = set(group['item_name'])
    for pair in combinations(items, 2):
        freq_counts[pair] += 1

## Frequency data frame

In [5]:
# Create frequency dataframe
freq_df = pd.DataFrame.from_dict(freq_counts, orient='index', columns=['frequency'])
freq_df.index = pd.MultiIndex.from_tuples(freq_df.index, names=['item1', 'item2'])
freq_df = freq_df.reset_index()

In [28]:
# Sort by frequency in descending order
freq_df = freq_df.sort_values('frequency', ascending=False)
freq_df
# freq_df.head(100)

,item1,item2,frequency
32,Bucatini Cacio e Pepe,Focaccia,7967
41,Bubbling Shrimp,Focaccia,6504
329,Focaccia,Spicy Meatballs,5814
228,Cavatelli Bolognese,Focaccia,5422
529,Wild Mushroom,Focaccia,5186
...,...,...,...
152861,Coramino High,Bubbling Shrimp,1
152862,Coramino High,Fire Pie,1
152863,Coramino High,Etta Old Fashioned,1
152864,Coramino High,"The ""etta"" Chopped Salad",1


## Price of the item per quantity

In [7]:
df = pd.read_csv( "D:\\Academics\\Datahack\\jetson-sample-data.csv")
df["item_per_quantity"] = df["price"]/df["quantity"]
pricelist = dict(zip(df["item_name"],df["item_per_quantity"]))

In [8]:
counts = df['item_name'].value_counts()

In [9]:
item = "Focaccia"
recommendations = freq_df[(freq_df['item1'] == item) | (freq_df['item2'] == item)]
recommendations = recommendations[['item1', 'item2', 'frequency']]
recommendations["probability"] = recommendations["frequency"]/counts[item]

In [10]:
comboBreakEven = []
for i in range(len(recommendations)):
    sumVal = pricelist[recommendations.iloc[i]["item1"]]+pricelist[recommendations.iloc[i]["item2"]]
    breakEven = sumVal - sumVal*0.6/100
    comboBreakEven.append(breakEven)
recommendations["Combo_break_even_price"]=comboBreakEven

In [11]:
recommendations

,item1,item2,frequency,probability,Combo_break_even_price
32,Bucatini Cacio e Pepe,Focaccia,7967,0.237729,28.826
41,Bubbling Shrimp,Focaccia,6504,0.194074,25.844
329,Focaccia,Spicy Meatballs,5814,0.173485,25.844
228,Cavatelli Bolognese,Focaccia,5422,0.161788,27.832
529,Wild Mushroom,Focaccia,5186,0.154746,31.808
...,...,...,...,...,...
108845,Cheese,Focaccia,1,0.000030,11.928
54685,GKS Cockjtail,Focaccia,1,0.000030,25.844
109316,Btl Ostatu Gran Reserva,Focaccia,1,0.000030,104.370
109082,Cakev Fee,Focaccia,1,0.000030,24.850


## Client Recommendation

In [14]:
client = df.iloc[1]["client_id"]

In [16]:
clientRec = df[df["client_id"]==client]

In [19]:
clientList = clientRec["item_name"]

In [20]:
clientList

0              Cosmonaut
1             Krombacher
2               Ahi Tuna
3       Little Gem Salad
4               Fire Pie
5             Fettuccine
6    Gl Cremant de Loire
Name: item_name, dtype: object

In [26]:
recommendations = freq_df[(freq_df['item1'].isin(clientList)) & (freq_df['item2'].isin(clientList))]
recommendations = recommendations[['item1', 'item2', 'frequency']]
recommendations["probability"] = recommendations["frequency"]/sum(recommendations["frequency"])

In [27]:
recommendations

,item1,item2,frequency,probability
1214,Little Gem Salad,Ahi Tuna,1015,0.224012
1336,Little Gem Salad,Fire Pie,739,0.163099
1071,Fire Pie,Ahi Tuna,392,0.086515
12535,Little Gem Salad,Krombacher,301,0.066431
2191,Gl Cremant de Loire,Little Gem Salad,229,0.050541
17849,Fire Pie,Krombacher,207,0.045685
8333,Krombacher,Ahi Tuna,179,0.039506
2198,Gl Cremant de Loire,Ahi Tuna,177,0.039064
16014,Cosmonaut,Little Gem Salad,134,0.029574
986,Fettuccine,Fire Pie,126,0.027808


In [29]:
len(df["date"].unique())

715

In [30]:
client

'ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3BfVmVudWVHcm91cENsaWVudBiAgPC1uszNCww'